# Tool Calls - Aufrufen eigener APIs

Dieses Notebook zeigt, wie man die Funktionsaufrufe mit dem Azure OpenAI Service verwendet. Funktionen ermöglichen es einem Aufrufer von Chatvervollständigungen, Fähigkeiten zu definieren, die das Modell verwenden kann, um seine
Funktionalität in externe Tools und Datenquellen zu erweitern.

Sie können mehr über Chat-Funktionen im OpenAI-Blog lesen: https://openai.com/blog/function-calling-and-other-api-updates


## Setup

Zunächst installieren wir die erforderlichen Abhängigkeiten und importieren die Bibliotheken, die wir verwenden werden.

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

#### Authentication

In [ ]:
client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-08-01-preview")

## Tool Calls

Nachdem die Einrichtung und Authentifizierung abgeschlossen sind, können wir nun Funktionen mit dem Azure OpenAI-Service verwenden. Das ist in folgende Schritte aufgeteilt:

1. Definieren Sie die Funktion(en)
2. Übergabe der Funktionsdefinition(en) an die Chatvervollständigungs-API
3. Aufruf der Funktion mit Argumenten aus der Antwort
4. Rückführung der Funktionsantwort in die Chatvervollständigungs-API


#### 1. Definieren der Funktion(en)

Es kann eine Liste von Funktionen definiert werden, die jeweils den Namen der Funktion, eine optionale Beschreibung und die Parameter, die die Funktion akzeptiert (beschrieben als JSON-Schema), enthalten.

In [ ]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

#### 2. Übergabe der Funktionsdefinition(en) an die Chatvervollständigungs-API

Jetzt können wir die Funktion an die Chatvervollständigungs-API übergeben. Wenn das Modell beschließt, die Funktion aufzurufen, wird ein `finish_reason` von "tool_calls" in die Auswahl eingefügt und die Details der aufzurufenden Funktion und ihre Argumente werden in der `message` enthalten sein. Optional können Sie das Schlüsselwortargument `tool_choice` setzen, um das Modell zum Aufruf einer bestimmten Funktion zu zwingen (z.B. `{"type": "function", "function": {"name": get_current_weather}}`). Standardmäßig ist dies auf `auto` gesetzt, so dass das Modell selbst entscheiden kann, ob es die Funktion aufrufen will oder nicht. 

In [ ]:
messages = [
    {"role": "system", "content": "Bitten Sie um Klärung, wenn eine Benutzeranfrage mehrdeutig ist."},
    {"role": "user", "content": "Wie ist das Wetter heute in Hamburg?"}
]

chat_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions,
)
print(chat_completion)

In [ ]:
# Anhängen der Antwort auf den Tool Call an die Nachrichtenliste
messages.append(chat_completion.choices[0].message)

#### 3. Aufruf einer Funktion mit den Argumenten aus der Antwort

Der Name des Funktionsaufrufs ist derjenige, der ursprünglich angegeben wurde, und die Argumente enthalten JSON, das dem in der Funktionsdefinition enthaltenen Schema entspricht.

In [ ]:
import json

def get_current_weather(request):
    """
    Demo function - do something useful here.
    """
    location = request.get("location")
    unit = request.get("unit")
    return {"temperature": "22", "unit": unit, "description": f"Wetter für {location}: Wolkig mit der aussicht auf Fleischbällchen"}

tool_call = chat_completion.choices[0].message.tool_calls[0].function
tool_call_id = chat_completion.choices[0].message.tool_calls[0].id
print(tool_call.name)
print(tool_call.arguments)

if tool_call.name == "get_current_weather":
    response = get_current_weather(json.loads(tool_call.arguments))

#### 4. Rückführung des Tool Calls in die Chatvervollständigungs-API

Die Antwort der Funktion sollte in eine neue Nachricht serialisiert werden, deren Rolle auf „tool“ gesetzt ist. Nun wird das Modell die Antwortdaten verwenden, um seine Antwort zu formulieren.

In [ ]:
messages.append(
    {
        "role": "tool",
        "name": "get_current_weather",
        "tool_call_id": tool_call_id,
        "content": json.dumps(response)
    }
)

function_completion = client.chat.completions.create(
    model=deployment,
    messages=messages    
)

print(function_completion.choices[0].message.content.strip())